In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
opt = SolverFactory('glpk').executable = 'D:\download\glpk-4.65\w64'

In [2]:
#Read generators and demand data as two dataframes  Desktop717data
df_NuclearData=pd.read_excel("Desktop/717/data/S1A3.xlsx",sheet_name="NuclearData",skiprows=1)
df_genData=pd.read_excel("Desktop/717/data/S1A3.xlsx",sheet_name="GeneratorsData",skiprows=1) #read data from generators tab and skip the first row
df_AverloadsData=pd.read_excel("Desktop/717/data/S1A3.xlsx",sheet_name="rooftopStorageAerageDemand2040",skiprows=1)
#df_PeakloadsData=pd.read_excel("Desktop/717/data/S1try1.xlsx",sheet_name="PeakLoadData",skiprows=1) 
df_linesData=pd.read_excel("Desktop/717/data/S1A3.xlsx",sheet_name="LinesData",skiprows=1) 
df_FossilFuel=pd.read_excel("Desktop/717/data/S1A3.xlsx",sheet_name="FossilFuelData",skiprows=1)
df_RenewableEnergy=pd.read_excel("Desktop/717/data/S1A3.xlsx",sheet_name="REData",skiprows=1)

In [3]:
#define the size of the sets
Nulear=len(df_NuclearData)
NumGens=len(df_genData)
NumNodes=len(df_AverloadsData)
NumLines=len(df_linesData)
NumFossilFuel=len(df_FossilFuel)
NumRenewableEnergy=len(df_RenewableEnergy)
#T=len(df_demandData)
print("We read data for", Nulear, "nuclear power generating units")
print("We read data for", NumGens, "no nuclear power generating units")
print("We read data for", NumNodes, "nodes")
print("We read data for", NumLines, "transmission lines")
print("We read data for", NumFossilFuel, "fossil fuel plants")
print("We read data for", NumRenewableEnergy, "renewable energy plants")

We read data for 6 nuclear power generating units
We read data for 18 no nuclear power generating units
We read data for 3 nodes
We read data for 3 transmission lines
We read data for 12 fossil fuel plants
We read data for 4 renewable energy plants


In [4]:
#Define Indexes
C=np.array([c for c in range(0,Nulear)])
G=np.array([g for g in range(0,NumGens)])  
N=np.array([n for n in range(0,NumNodes)])
L=np.array([l for l in range(0,NumLines)])
F=np.array([f for f in range(0,NumFossilFuel)])
R=np.array([r for r in range(0,NumRenewableEnergy)])

In [18]:
df_FossilFuel.columns

Index(['GenName', 'Node', 'FixedCost', 'SDCost', 'SUCost', 'VarCost', 'MaxGen',
       'PSpringMaxGen', 'PSummerMaxGen', 'PAutumnMaxGen', 'PWinterMaxGen',
       'OSpringMaxGen', 'OSummerMaxGen', 'OAutumnMaxGen', 'OWinterMaxGen',
       'MinGen', 'RampDown', 'RampSD', 'RampSU', 'RampUp', 'MinUpTime',
       'MinDownTime', 'ReqUp', 'ReqDown', 'InitialStatus', 'InitialGen',
       'Plant Name', 'Fuel Type', 'Average Emission Rate'],
      dtype='object')

In [19]:
sum(df_FossilFuel.MaxGen)

11078.500000000002

In [5]:
##Declare the generator's parameters and assign them the data we read before
NVarCost=df_NuclearData.loc[:,'NVarCost'].to_numpy()
VarCost=df_genData.loc[:,'VarCost'].to_numpy()
# generation capacity of peak hour for each seasons
PSpringPMax=df_genData.loc[:,'PSpringMaxGen'].to_numpy()
PSummerPMax=df_genData.loc[:,'PSummerMaxGen'].to_numpy()
PFallPMax=df_genData.loc[:,'PAutumnMaxGen'].to_numpy()
PWinterPMax=df_genData.loc[:,'PWinterMaxGen'].to_numpy()
# generation capacity of off-peak hour for each seasons
OSpringPMax=df_genData.loc[:,'OSpringMaxGen'].to_numpy()
OSummerPMax=df_genData.loc[:,'OSummerMaxGen'].to_numpy()
OFallPMax=df_genData.loc[:,'OAutumnMaxGen'].to_numpy()
OWinterPMax=df_genData.loc[:,'OWinterMaxGen'].to_numpy()
# nuclear generation capacity for each seasons
NSpringPMax=df_NuclearData.loc[:,'NSpringMaxGen'].to_numpy()
NSummerPMax=df_NuclearData.loc[:,'NSummerMaxGen'].to_numpy()
NFallPMax=df_NuclearData.loc[:,'NAutumnMaxGen'].to_numpy()
NWinterPMax=df_NuclearData.loc[:,'NWinterMaxGen'].to_numpy()
#emission rate of each generator
GEmissionRate=df_genData.loc[:,'Average Emission Rate'].to_numpy()
#other attributes
PMin=df_genData.loc[:,'MinGen'].to_numpy()
NGenNode=df_NuclearData.loc[:,'NNode'].to_numpy()
GenNode=df_genData.loc[:,'Node'].to_numpy()

In [8]:
NumLines #In the data we defined 3 lines as: L1-2, L2-3, and L3-1.
#We could have defined them in other way, but this is convenient for the KVL equation

3

In [6]:
##Declare the transmission lines' parameters and assign them the data we read before
NodeFrom=df_linesData.loc[:,'NodeFrom'].to_numpy()
NodeTo=df_linesData.loc[:,'NodeTo'].to_numpy()
LineReactance=df_linesData.loc[:,'Reactance'].to_numpy()
LineCapacity=df_linesData.loc[:,'Capacity'].to_numpy()

In [7]:
df_AverloadsData.columns

Index(['LoadName', 'Node', 'PeakWinter', 'PeakSpring', 'PeakSummer',
       'PeakFall', 'Off-PeakWinter', 'Off-PeakSpring', 'Off-PeakSummer',
       'Off-PeakFall'],
      dtype='object')

In [7]:
ALoadBus=df_AverloadsData.loc[:,'Node'].to_numpy( )
#Demand of peak hours and off-peak hour in each seasons
APeakSpringDemand=df_AverloadsData.loc[:,'PeakSpring'].to_numpy( )
AOffPeakSpringDemand=df_AverloadsData.loc[:,'Off-PeakSpring'].to_numpy( )
APeakSummerDemand=df_AverloadsData.loc[:,'PeakSummer'].to_numpy( )
AOffPeakSummerDemand=df_AverloadsData.loc[:,'Off-PeakSummer'].to_numpy( )
APeakFallDemand=df_AverloadsData.loc[:,'PeakFall'].to_numpy( )
AOffPeakFallDemand=df_AverloadsData.loc[:,'Off-PeakFall'].to_numpy( )
APeakWinterDemand=df_AverloadsData.loc[:,'PeakWinter'].to_numpy( )
AOffPeakWinterDemand=df_AverloadsData.loc[:,'Off-PeakWinter'].to_numpy( )


In [8]:
# define a function indicatorMatrix that creates indicator matrices to relate generators with their node
# and relate lines with the nodes
#dataRows is a column vector of NumRows rows. Each element says the column where this row should be a one in the final matrix
def IndicatorMatrix(NumRows, NumCols, dataRows):
    matrix= np.zeros((NumRows, NumCols),dtype=int)
    for i in range(0,NumRows):
        matrix[i,dataRows[i]-1]=1
    return matrix

In [9]:
#Create indicator matrices using the function defined above
#NuclearInBus is a matrix of C rows and N columns that has 1 in the position c,n if generator g is AT node N, 0 otherwise
NuclearInBus=IndicatorMatrix(Nulear,NumNodes,NGenNode)
#GeneratorInBus is a matrix of G rows and N columns that has 1 in the position g,n if generator g is AT node N, 0 otherwise
GeneratorInBus=IndicatorMatrix(NumGens,NumNodes,GenNode)
#LineFromBus is an indicator matrix of L rows and N columns that has 1 in the position l,n if line l departs FROM node N, 0 otherwise
LineFromBus=IndicatorMatrix(NumLines,NumNodes,NodeFrom)
#LineToBus is an indicator matrix of L rows and N columns that has 1 in the position l,n if line l arrives TO node N, 0 otherwise
LineToBus=IndicatorMatrix(NumLines,NumNodes,NodeTo)

In [25]:
NuclearInBus

array([[1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

Average Winter peak hour

In [15]:
sum(APeakWinterDemand)

25513.031621350317

In [10]:
#UC MODEL of Average winter peak hour   
def APeakWinterED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.F=Set(initialize=F)
    m.p=Var(m.G, bounds = (0,20000))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-5000,5000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G)+ sum(NVarCost[c]*NWinterPMax[c] for c in C), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PWinterPMax[g])
    #Spinning reserves requirement as a fraction of 3% of net demand
    m.SpinningReserve1=Constraint(m.F, rule=lambda m, f: sum(df_FossilFuel.MaxGen)-sum(m.p[f] for f in F)>=0.03*sum(APeakWinterDemand))
    # refer Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G)+sum(NuclearInBus[c,n]*NWinterPMax[c] for c in C)==APeakWinterDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m



In [11]:
m=APeakWinterED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 167146.423
  Upper bound: 167146.423
  Number of objectives: 1
  Number of constraints: 41
  Number of variables: 22
  Number of nonzeros: 196
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.05154132843017578
# ----------------------------------------------------------
#   Solution Information
# -----------------------

In [12]:
# Total emission in this hour
# create an empty DataFrame with the appropriate column names
dfeachgen = pd.DataFrame(columns=["Generator", "Power Output"])

for g in m.G:
    dfeachgen.loc[g, "Generator"] = g
    dfeachgen.loc[g, "Power Output"] = m.p[g].value


TotalEmission= dfeachgen['Power Output']*df_genData['Average Emission Rate']

EM=sum(TotalEmission)* 0.000453592

print('{:1f} metric ton'.format(EM))

0.000000 metric ton


In [13]:
print('SOLUTION-Average winter peak')
print('The total system cost is = $',m.system_cost())

print('Generation in MW')
for g in G:
        print("non nuclear generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))
        


SOLUTION-Average winter peak
The total system cost is = $ 167146.42299999998
Generation in MW
non nuclear generator  1 = 0.00 
non nuclear generator  2 = 0.00 
non nuclear generator  3 = 0.00 
non nuclear generator  4 = 0.00 
non nuclear generator  5 = 0.00 
non nuclear generator  6 = 0.00 
non nuclear generator  7 = 0.00 
non nuclear generator  8 = 0.00 
non nuclear generator  9 = 0.00 
non nuclear generator 10 = 0.00 
non nuclear generator 11 = 0.00 
non nuclear generator 12 = 0.00 
non nuclear generator 13 = 13800.47 
non nuclear generator 14 = 490.84 
non nuclear generator 15 = 490.84 
non nuclear generator 16 = 368.00 
non nuclear generator 17 = 0.00 
non nuclear generator 18 = 0.00 
Flow on transmission lines in MW
line  1 = 3347.54 
line  2 = -606.98 
line  3 = -2740.55 
LMPs in $/MWh
Node  1 = 0.00
Node  2 = 0.00
Node  3 = 0.00


Average Winter off peak hour

In [14]:
#UC MODEL of Average winter off peak hour   
def AOffPeakWinterED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.F=Set(initialize=F)
    m.p=Var(m.G, bounds = (0,20000))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-5000,5000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G)+ sum(NVarCost[c]*NWinterPMax[c] for c in C), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= OWinterPMax[g])
    #Spinning reserves requirement as a fraction of 3% of net demand
    m.SpinningReserve1=Constraint(m.F, rule=lambda m, f: sum(df_FossilFuel.MaxGen)-sum(m.p[f] for f in F)>=0.03*sum(AOffPeakWinterDemand))
    #m.MaxEmission= Constraint(m.G, rule=lambda m, g:  sum(m.p[g]* (GEmissionRate[g])for g in m.G) <= 6000)#maybe put parameter to replace exact number
    # refer Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)
   
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)
                                            -sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G)
                                            +sum(NuclearInBus[c,n]*NWinterPMax[c] for c in C)
                                            == AOffPeakWinterDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [15]:
m= AOffPeakWinterED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 167146.423
  Upper bound: 167146.423
  Number of objectives: 1
  Number of constraints: 41
  Number of variables: 22
  Number of nonzeros: 196
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04832196235656738
# ----------------------------------------------------------
#   Solution Information
# -----------------------

In [16]:
# Total emission in this hour
# create an empty DataFrame with the appropriate column names
dfeachgen = pd.DataFrame(columns=["Generator", "Power Output"])

for g in m.G:
    dfeachgen.loc[g, "Generator"] = g
    dfeachgen.loc[g, "Power Output"] = m.p[g].value


TotalEmission= dfeachgen['Power Output']*df_genData['Average Emission Rate']

EM=sum(TotalEmission)* 0.000453592

print('{:1f} metric ton'.format(EM))

0.000000 metric ton


In [17]:
print('SOLUTION-Average winter off peak')
print('The total system cost is = $',m.system_cost())

print('Generation in MW')
for g in G:
        print("non nuclear generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))
        

SOLUTION-Average winter off peak
The total system cost is = $ 167146.42299999998
Generation in MW
non nuclear generator  1 = 0.00 
non nuclear generator  2 = 0.00 
non nuclear generator  3 = 0.00 
non nuclear generator  4 = 0.00 
non nuclear generator  5 = 0.00 
non nuclear generator  6 = 0.00 
non nuclear generator  7 = 0.00 
non nuclear generator  8 = 0.00 
non nuclear generator  9 = 0.00 
non nuclear generator 10 = 0.00 
non nuclear generator 11 = 0.00 
non nuclear generator 12 = 0.00 
non nuclear generator 13 = 6721.58 
non nuclear generator 14 = 0.00 
non nuclear generator 15 = 0.00 
non nuclear generator 16 = 384.00 
non nuclear generator 17 = 0.00 
non nuclear generator 18 = 0.00 
Flow on transmission lines in MW
line  1 = 2893.42 
line  2 = -537.15 
line  3 = -2356.28 
LMPs in $/MWh
Node  1 = 0.00
Node  2 = 0.00
Node  3 = 0.00


Average Summer peak hour

In [18]:
#UC MODEL of Average summer peak hour   
def APeakSummerED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.F=Set(initialize=F)
    m.p=Var(m.G, bounds = (0,20000))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-5000,5000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G)+ sum(NVarCost[c]*NSummerPMax[c] for c in C), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PSummerPMax[g])
     #Spinning reserves requirement as a fraction of 3% of net demand
    m.SpinningReserve1=Constraint(m.F, rule=lambda m, f: sum(df_FossilFuel.MaxGen)-sum(m.p[f] for f in F)>=0.03*sum(APeakSummerDemand))
    #m.MaxEmission= Constraint(m.G, rule=lambda m, g:  sum(m.p[g]* (GEmissionRate[g])for g in m.G) <= 6000)#maybe put parameter to replace exact number
    # refer Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)
   
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)
                                            -sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G)
                                            +sum(NuclearInBus[c,n]*NSummerPMax[c] for c in C)
                                            == APeakSummerDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [19]:
m= APeakSummerED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 232582.786661402
  Upper bound: 232582.786661402
  Number of objectives: 1
  Number of constraints: 41
  Number of variables: 22
  Number of nonzeros: 196
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.03609609603881836
# ----------------------------------------------------------
#   Solution Information
# -----------

In [20]:
# Total emission in this hour
# create an empty DataFrame with the appropriate column names
dfeachgen = pd.DataFrame(columns=["Generator", "Power Output"])

for g in m.G:
    dfeachgen.loc[g, "Generator"] = g
    dfeachgen.loc[g, "Power Output"] = m.p[g].value


TotalEmission= dfeachgen['Power Output']*df_genData['Average Emission Rate']

EM=sum(TotalEmission)* 0.000453592

print('{:1f} metric ton'.format(EM))

1211.648717 metric ton


In [21]:
print('SOLUTION-Average summer peak')
print('The total system cost is = $',m.system_cost())

print('Generation in MW')
for g in G:
        print("non nuclear generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))
        

SOLUTION-Average summer peak
The total system cost is = $ 232582.78666140238
Generation in MW
non nuclear generator  1 = 0.00 
non nuclear generator  2 = 697.90 
non nuclear generator  3 = 313.36 
non nuclear generator  4 = 0.00 
non nuclear generator  5 = 0.00 
non nuclear generator  6 = 0.00 
non nuclear generator  7 = 0.00 
non nuclear generator  8 = 0.00 
non nuclear generator  9 = 1248.01 
non nuclear generator 10 = 730.00 
non nuclear generator 11 = 0.00 
non nuclear generator 12 = 0.00 
non nuclear generator 13 = 17233.14 
non nuclear generator 14 = 504.88 
non nuclear generator 15 = 504.88 
non nuclear generator 16 = 1020.00 
non nuclear generator 17 = 0.00 
non nuclear generator 18 = 0.00 
Flow on transmission lines in MW
line  1 = 5000.00 
line  2 = -888.38 
line  3 = -4111.62 
LMPs in $/MWh
Node  1 = 22.00
Node  2 = 24.00
Node  3 = 23.00


Average Summer off peak hour

In [22]:
#UC MODEL of Average summer off peak hour   
def AOffPeakSummerED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.F=Set(initialize=F)
    m.p=Var(m.G, bounds = (0,20000))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-5000,5000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G)+ sum(NVarCost[c]*NSummerPMax[c] for c in C), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= OSummerPMax[g])
     #Spinning reserves requirement as a fraction of 3% of net demand
    m.SpinningReserve1=Constraint(m.F, rule=lambda m, f: sum(df_FossilFuel.MaxGen)-sum(m.p[f] for f in F)>=0.03*sum(AOffPeakSummerDemand))
    #m.MaxEmission= Constraint(m.G, rule=lambda m, g:  sum(m.p[g]* (GEmissionRate[g])for g in m.G) <= 6000)#maybe put parameter to replace exact number
    # refer Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)
   
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)
                                            -sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G)
                                            +sum(NuclearInBus[c,n]*NSummerPMax[c] for c in C)
                                            == AOffPeakSummerDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [23]:
m= AOffPeakSummerED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 167146.423
  Upper bound: 167146.423
  Number of objectives: 1
  Number of constraints: 41
  Number of variables: 22
  Number of nonzeros: 196
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.048392534255981445
# ----------------------------------------------------------
#   Solution Information
# ----------------------

In [24]:
# Total emission in this hour
# create an empty DataFrame with the appropriate column names
dfeachgen = pd.DataFrame(columns=["Generator", "Power Output"])

for g in m.G:
    dfeachgen.loc[g, "Generator"] = g
    dfeachgen.loc[g, "Power Output"] = m.p[g].value


TotalEmission= dfeachgen['Power Output']*df_genData['Average Emission Rate']

EM=sum(TotalEmission)* 0.000453592

print('{:1f} metric ton'.format(EM))

0.000000 metric ton


In [25]:
print('SOLUTION-Average summer off peak')
print('The total system cost is = $',m.system_cost())

print('Generation in MW')
for g in G:
        print("non nuclear generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-Average summer off peak
The total system cost is = $ 167146.42299999998
Generation in MW
non nuclear generator  1 = 0.00 
non nuclear generator  2 = 0.00 
non nuclear generator  3 = 0.00 
non nuclear generator  4 = 0.00 
non nuclear generator  5 = 0.00 
non nuclear generator  6 = 0.00 
non nuclear generator  7 = 0.00 
non nuclear generator  8 = 0.00 
non nuclear generator  9 = 0.00 
non nuclear generator 10 = 0.00 
non nuclear generator 11 = 0.00 
non nuclear generator 12 = 0.00 
non nuclear generator 13 = 10357.27 
non nuclear generator 14 = 0.00 
non nuclear generator 15 = 0.00 
non nuclear generator 16 = 1231.00 
non nuclear generator 17 = 0.00 
non nuclear generator 18 = 0.00 
Flow on transmission lines in MW
line  1 = 4478.87 
line  2 = -968.90 
line  3 = -3509.97 
LMPs in $/MWh
Node  1 = 0.00
Node  2 = 0.00
Node  3 = 0.00


# Average Fall peak hour

In [26]:
#UC MODEL of Average Fall peak hour   
def APeakFallED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.F=Set(initialize=F)
    m.p=Var(m.G, bounds = (0,20000))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-5000,5000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G)+ sum(NVarCost[c]*NFallPMax[c] for c in C), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PFallPMax[g])
     #Spinning reserves requirement as a fraction of 3% of net demand
    m.SpinningReserve1=Constraint(m.F, rule=lambda m, f: sum(df_FossilFuel.MaxGen)-sum(m.p[f] for f in F)>=0.03*sum(APeakFallDemand))
    #m.MaxEmission= Constraint(m.G, rule=lambda m, g:  sum(m.p[g]* (GEmissionRate[g])for g in m.G) <= 6000)#maybe put parameter to replace exact number
    # refer Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)
   
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)
                                            -sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G)
                                            +sum(NuclearInBus[c,n]*NFallPMax[c] for c in C)
                                            == APeakFallDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [27]:
m= APeakFallED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 168514.62576777
  Upper bound: 168514.62576777
  Number of objectives: 1
  Number of constraints: 41
  Number of variables: 22
  Number of nonzeros: 196
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04678964614868164
# ----------------------------------------------------------
#   Solution Information
# -------------

In [28]:
# Total emission in this hour
# create an empty DataFrame with the appropriate column names
dfeachgen = pd.DataFrame(columns=["Generator", "Power Output"])

for g in m.G:
    dfeachgen.loc[g, "Generator"] = g
    dfeachgen.loc[g, "Power Output"] = m.p[g].value


TotalEmission= dfeachgen['Power Output']*df_genData['Average Emission Rate']

EM=sum(TotalEmission)* 0.000453592

print('{:1f} metric ton'.format(EM))

26.731453 metric ton


In [29]:
print('SOLUTION-Average Fall peak')
print('The total system cost is = $',m.system_cost())

print('Generation in MW')
for g in G:
        print("non nuclear generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-Average Fall peak
The total system cost is = $ 168514.6257677696
Generation in MW
non nuclear generator  1 = 0.00 
non nuclear generator  2 = 72.01 
non nuclear generator  3 = 0.00 
non nuclear generator  4 = 0.00 
non nuclear generator  5 = 0.00 
non nuclear generator  6 = 0.00 
non nuclear generator  7 = 0.00 
non nuclear generator  8 = 0.00 
non nuclear generator  9 = 0.00 
non nuclear generator 10 = 0.00 
non nuclear generator 11 = 0.00 
non nuclear generator 12 = 0.00 
non nuclear generator 13 = 15013.76 
non nuclear generator 14 = 439.86 
non nuclear generator 15 = 439.86 
non nuclear generator 16 = 676.00 
non nuclear generator 17 = 0.00 
non nuclear generator 18 = 0.00 
Flow on transmission lines in MW
line  1 = 3917.24 
line  2 = -759.36 
line  3 = -3157.88 
LMPs in $/MWh
Node  1 = 19.00
Node  2 = 19.00
Node  3 = 19.00


Average Fall off peak hour

In [30]:
#UC MODEL of Average Fall off peak hour   
def AOffPeakFallED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.F=Set(initialize=F)
    m.p=Var(m.G, bounds = (0,20000))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-5000,5000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G)+ sum(NVarCost[c]*NFallPMax[c] for c in C), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= OFallPMax[g])
     #Spinning reserves requirement as a fraction of 3% of net demand
    m.SpinningReserve1=Constraint(m.F, rule=lambda m, f: sum(df_FossilFuel.MaxGen)-sum(m.p[f] for f in F)>=0.03*sum(AOffPeakFallDemand))
    #m.MaxEmission= Constraint(m.G, rule=lambda m, g:  sum(m.p[g]* (GEmissionRate[g])for g in m.G) <= 6000)#maybe put parameter to replace exact number
    # refer Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)
   
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)
                                            -sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G)
                                            +sum(NuclearInBus[c,n]*NFallPMax[c] for c in C)
                                            == AOffPeakFallDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [31]:
m= AOffPeakFallED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 167146.423
  Upper bound: 167146.423
  Number of objectives: 1
  Number of constraints: 41
  Number of variables: 22
  Number of nonzeros: 196
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.03807353973388672
# ----------------------------------------------------------
#   Solution Information
# -----------------------

In [32]:
# Total emission in this hour
# create an empty DataFrame with the appropriate column names
dfeachgen = pd.DataFrame(columns=["Generator", "Power Output"])

for g in m.G:
    dfeachgen.loc[g, "Generator"] = g
    dfeachgen.loc[g, "Power Output"] = m.p[g].value


TotalEmission= dfeachgen['Power Output']*df_genData['Average Emission Rate']

EM=sum(TotalEmission)* 0.000453592

print('{:1f} metric ton'.format(EM))

0.000000 metric ton


In [33]:
print('SOLUTION-Average Fall off peak')
print('The total system cost is = $',m.system_cost())

print('Generation in MW')
for g in G:
        print("non nuclear generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-Average Fall off peak
The total system cost is = $ 167146.42299999998
Generation in MW
non nuclear generator  1 = 0.00 
non nuclear generator  2 = 0.00 
non nuclear generator  3 = 0.00 
non nuclear generator  4 = 0.00 
non nuclear generator  5 = 0.00 
non nuclear generator  6 = 0.00 
non nuclear generator  7 = 0.00 
non nuclear generator  8 = 0.00 
non nuclear generator  9 = 0.00 
non nuclear generator 10 = 0.00 
non nuclear generator 11 = 0.00 
non nuclear generator 12 = 0.00 
non nuclear generator 13 = 8826.20 
non nuclear generator 14 = 0.00 
non nuclear generator 15 = 0.00 
non nuclear generator 16 = 710.00 
non nuclear generator 17 = 0.00 
non nuclear generator 18 = 0.00 
Flow on transmission lines in MW
line  1 = 3797.51 
line  2 = -726.83 
line  3 = -3070.68 
LMPs in $/MWh
Node  1 = 0.00
Node  2 = 0.00
Node  3 = 0.00


In [34]:
#UC MODEL of Average Spring peak hour   
def APeakSpringED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.F=Set(initialize=F)
    m.p=Var(m.G, bounds = (0,20000))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-5000,5000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G)+ sum(NVarCost[c]*NSpringPMax[c] for c in C), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PSpringPMax[g])
     #Spinning reserves requirement as a fraction of 3% of net demand
    m.SpinningReserve1=Constraint(m.F, rule=lambda m, f: sum(df_FossilFuel.MaxGen)-sum(m.p[f] for f in F)>=0.03*sum(APeakSpringDemand))
    #m.MaxEmission= Constraint(m.G, rule=lambda m, g:  sum(m.p[g]* (GEmissionRate[g])for g in m.G) <= 6000)#maybe put parameter to replace exact number
    # refer Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)
   
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)
                                            -sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G)
                                            +sum(NuclearInBus[c,n]*NSpringPMax[c] for c in C)
                                            == APeakSpringDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [35]:
m= APeakSpringED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 107743.092901691
  Upper bound: 107743.092901691
  Number of objectives: 1
  Number of constraints: 41
  Number of variables: 22
  Number of nonzeros: 196
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.05730032920837402
# ----------------------------------------------------------
#   Solution Information
# -----------

In [36]:
# Total emission in this hour
# create an empty DataFrame with the appropriate column names
dfeachgen = pd.DataFrame(columns=["Generator", "Power Output"])

for g in m.G:
    dfeachgen.loc[g, "Generator"] = g
    dfeachgen.loc[g, "Power Output"] = m.p[g].value


TotalEmission= dfeachgen['Power Output']*df_genData['Average Emission Rate']

EM=sum(TotalEmission)* 0.000453592

print('{:1f} metric ton'.format(EM))

449.026093 metric ton


In [37]:
print('SOLUTION-Average Spring peak')
print('The total system cost is = $',m.system_cost())

print('Generation in MW')
for g in G:
        print("non nuclear generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-Average Spring peak
The total system cost is = $ 107743.09290169086
Generation in MW
non nuclear generator  1 = 0.00 
non nuclear generator  2 = 0.00 
non nuclear generator  3 = 0.00 
non nuclear generator  4 = 0.00 
non nuclear generator  5 = 0.00 
non nuclear generator  6 = 0.00 
non nuclear generator  7 = 0.00 
non nuclear generator  8 = 0.00 
non nuclear generator  9 = 1007.08 
non nuclear generator 10 = 0.00 
non nuclear generator 11 = 0.00 
non nuclear generator 12 = 0.00 
non nuclear generator 13 = 18401.68 
non nuclear generator 14 = 559.68 
non nuclear generator 15 = 559.68 
non nuclear generator 16 = 715.00 
non nuclear generator 17 = 0.00 
non nuclear generator 18 = 0.00 
Flow on transmission lines in MW
line  1 = 5000.00 
line  2 = -503.91 
line  3 = -4496.09 
LMPs in $/MWh
Node  1 = 0.00
Node  2 = 24.00
Node  3 = 12.00


Average Spring off peak hour

In [42]:
#UC MODEL of Average Spring off peak hour   
def AOffPeakSpringED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.F=Set(initialize=F)
    m.p=Var(m.G, bounds = (0,20000))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-5000,5000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G)+ sum(NVarCost[c]*NSpringPMax[c] for c in C), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= OSpringPMax[g])
     #Spinning reserves requirement as a fraction of 3% of net demand
    m.SpinningReserve1=Constraint(m.F, rule=lambda m, f: sum(df_FossilFuel.MaxGen)-sum(m.p[f] for f in F)>=0.03*sum(AOffPeakSpringDemand))
    #m.MaxEmission= Constraint(m.G, rule=lambda m, g:  sum(m.p[g]* (GEmissionRate[g])for g in m.G) <= 2000)#maybe put parameter to replace exact number
    # refer Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)
   
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)
                                            -sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G)
                                            +sum(NuclearInBus[c,n]*NSpringPMax[c] for c in C)
                                            == AOffPeakSpringDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [43]:
m= AOffPeakSpringED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 141038.51365909
  Upper bound: 141038.51365909
  Number of objectives: 1
  Number of constraints: 41
  Number of variables: 22
  Number of nonzeros: 196
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04250788688659668
# ----------------------------------------------------------
#   Solution Information
# -------------

In [44]:
# Total emission in this hour
# create an empty DataFrame with the appropriate column names
dfeachgen = pd.DataFrame(columns=["Generator", "Power Output"])

for g in m.G:
    dfeachgen.loc[g, "Generator"] = g
    dfeachgen.loc[g, "Power Output"] = m.p[g].value


TotalEmission= dfeachgen['Power Output']*df_genData['Average Emission Rate']

EM=sum(TotalEmission)* 0.000453592

print('{:1f} metric ton'.format(EM))

1065.110672 metric ton


In [45]:
print('SOLUTION-Average Spring off peak')
print('The total system cost is = $',m.system_cost())

print('Generation in MW')
for g in G:
        print("non nuclear generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-Average Spring off peak
The total system cost is = $ 141038.5136590904
Generation in MW
non nuclear generator  1 = 0.00 
non nuclear generator  2 = 307.40 
non nuclear generator  3 = 0.00 
non nuclear generator  4 = 0.00 
non nuclear generator  5 = 0.00 
non nuclear generator  6 = 0.00 
non nuclear generator  7 = 0.00 
non nuclear generator  8 = 0.00 
non nuclear generator  9 = 1512.28 
non nuclear generator 10 = 730.00 
non nuclear generator 11 = 0.00 
non nuclear generator 12 = 0.00 
non nuclear generator 13 = 11000.00 
non nuclear generator 14 = 0.00 
non nuclear generator 15 = 0.00 
non nuclear generator 16 = 796.00 
non nuclear generator 17 = 0.00 
non nuclear generator 18 = 0.00 
Flow on transmission lines in MW
line  1 = 5000.00 
line  2 = -1179.84 
line  3 = -3820.16 
LMPs in $/MWh
Node  1 = 19.00
Node  2 = 24.00
Node  3 = 21.50
